In [1]:
import xml.etree.ElementTree as ET
import re
import unicodedata
from docx import Document
from lxml import etree
import pandas as pd
import os
from win32com import client


In [2]:
with open('01 Hosea App II Final.doc.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()
    txt = file
lines

['\ufeffChapter 1\n',
 '1 מלכי] b. Pesaḥim 87a מלךI\n',
 '2 לֵךְ] b. Pesaḥimms 87a, S.Eli.Z 9 (187) >I\n',
 '3 לו] b. Pesaḥim 87b >I III IV\n',
 '4\n',
 '5\n',
 '6 עוד1] b. Pesaḥimmss 87b >I III IV \n',
 'לו] b. Pesaḥimmss 87b יי אליוI\n',
 '7 אלהיהם] 4QXIId אלהים֯ (!)[1]\n',
 '8 ותהר] b. Pesaḥimmss 87b + עודI\n',
 '9 ויאמר] b. Pesaḥimmss 87b + יי אליוI | b. Pesaḥimms 87b + יי\n',
 '\n',
 '\n',
 'Chapter 2\n',
 '1 יֵאָמֵר] 4QXIId יומ[ר[2]\n',
 '2\n',
 '3\n',
 '4 ריבו2] 4QXIIg ר֯[י]ב֗\n',
 'מ(פניה)] 4QXIId מבי[ן[3] \n',
 '5 כמדבר ושתה] 4QXIIg (pm) >[4]  \n',
 '6\n',
 '7\n',
 '8 שך] b. R.haŠana 23b, LamR Buber 1 (40b) סךIII IV\n',
 'וגדרתי את גדרה] 4QpHosa > (lem)[5]\n',
 '9\n',
 '10 (הרביתי) לה] 4QpHosa > (lem)\n',
 'וזהב] 4QpHosa (pm) + [..] (lem)\n',
 '11 לכסות] 4QpHosa מלכסות (lem)[6]I\n',
 '12\n',
 '13 מועדה] 4QpHosa מועדיה (lem)[7]\n',
 '14 אתנה המה] 4QpHosa אתנם הם (lem)[8]\n',
 '15 ופקדתי – ותעד] 4QXIIg [↔][9]\n',
 '16 והלכתיה – לבה] 4QXIIg [↔][10]\n',
 '17\n',
 '18 תקראי1] b. Pe

In [ ]:
########## render apparatus 3 ##############

In [ ]:
#new logic
#1. split into lemma and app_entry. process each separately
# lemma: split into number range and lemma range. 
#numbers include \d+'-', lemmas need to include potential k\q attribute, and word number.
# app_entry: splits into mss., reading, comments, cross-reference.
# 
#to do: 
#1. get_verse, if verse is empty. same for get_witness (if its inside a comma)
#2. QA: "איסירם] 30 89 איסרם | 96 (pm) אייסרם, (sm) אייסירים | 150 (pm) אסירים" (when split on comma allow for witness completion)
# maybe run a post_processing function that will add .. if needed to the sigla, and the verse and witness if their missing.
# get chapter of verse

In [36]:
##### 
def compile_and_display_entries(entries):
    previous_verse = None
    current_chapter = None
    compiled_results = []
    
    for entry in entries:
        chapter_match = re.search(r'Chapter (\d+)', entry)
        if chapter_match:
            current_chapter = chapter_match.group(1)
            continue
        elif entry.strip() and not entry.strip().isdigit():
            clean_entry = entry.strip()
            lemma_dict, decoded_entries, used_verse = process_full_entry(clean_entry, current_chapter, previous_verse)
            previous_verse = used_verse

            # Combine the lemma_dict and decoded_entries for display
            entry_result = {
                #'Chapter': current_chapter,
                'Entry': clean_entry,
                'Lemma Info': lemma_dict,
                'Decoded Entries': []
            }
            last_witness_second_group = None
            for i, decoded_entries_list in enumerate(decoded_entries):
                for j, decoded_entry in enumerate(decoded_entries_list):
                    variant_type = "Variant" if j == 0 else "Related Variant"
                    variant_dict = {'Type': variant_type, 'Details': []}
                    for item in decoded_entry:
                        if isinstance(item, dict) and 'Witnesses' in item:
                            # Update witnesses with last non-empty second group if needed
                            updated_witnesses = []
                            for witness in item['Witnesses']:
                                if witness[1] == '' and last_witness_second_group:
                                    updated_witnesses.append((witness[0], last_witness_second_group, witness[2]))
                                else:
                                    updated_witnesses.append(witness)
                                    if witness[1]:
                                        last_witness_second_group = witness[1]
                            # Apply post-processing to witnesses
                            item['Witnesses'] = post_process_witnesses(updated_witnesses)
                        variant_dict['Details'].append(item)
                    entry_result['Decoded Entries'].append(variant_dict)
            compiled_results.append(entry_result)

    # Display the results in a structured format
    for result in compiled_results:
        print(f"Entry:")
        print(f"{result['Entry']}")
        print("Lemma Info:")
        for key, value in result['Lemma Info'].items():
            print(f"  {key}: {value}")
        print("Decoded Entries:")
        for variant in result['Decoded Entries']:
            print(f"  {variant['Type']}:")
            for detail in variant['Details']:
                if isinstance(detail, dict):
                    for key, value in detail.items():
                        print(f"    {key}: {value}")
                else:
                    print(f"    {detail}")
        print("-" * 50)  # Separator line
    
    return compiled_results

# Example usage
test = compile_and_display_entries(lines)

IndexError: string index out of range

In [34]:
def extract_footnotes(text):
    """
    Extracts footnotes from the text, denoted by numbers in square brackets.

    Args:
        text (str): The text from which to extract footnotes.

    Returns:
        tuple: A tuple containing the cleaned text (without footnotes) and a list of footnotes.
    """
    # Regex to match footnotes in the format [number]
    footnote_pattern = re.compile(r'\[(\d+)\]')
    
    # Find all footnotes
    footnotes = footnote_pattern.findall(text)
    
    # Remove footnotes from the text
    cleaned_text = footnote_pattern.sub('', text)
    
    return cleaned_text, footnotes

def split_full_entry(text):
    """
    Splits a full entry into lemma and part_entry based on the first occurrence of ']'.
    
    Args:
        text (str): The full entry text to be split.

    Returns:
        tuple: A tuple containing the lemma and part_entry.
               If the split is not possible, returns the entire text as lemma and an empty string as part_entry.
    """
    sliced_entry = text.split(sep=']', maxsplit=1)
    if len(sliced_entry) == 2:
        lemma, part_entry = sliced_entry
    else:
        lemma, part_entry = text, ""
    return lemma, part_entry


In [35]:
def extract_manuscript_witnesses(text):
    """
    Extracts manuscript witnesses ("ms" or "mss") from the text.

    Args:
        text (str): The text from which to extract manuscript witnesses.

    Returns:
        list: A list of manuscript witnesses.
    """
    pattern = re.compile(r'\b(ms|mss)\b', re.IGNORECASE)
    manuscript_witnesses = pattern.findall(text)
    
    # Clean the text by removing manuscript witnesses
    cleaned_text = pattern.sub('', text)
    
    return manuscript_witnesses

# Adjust other functions to integrate the manuscript extraction
def process_entry(entry):
    """
    Processes a textual entry to extract witnesses, readings, and footnotes.

    Args:
        entry (str): The entry text to be processed.

    Returns:
        list: A list containing processed witnesses, readings, footnotes, and manuscript witnesses.
    """
    # Extract and remove footnotes
    clean_entry, footnotes = extract_footnotes(entry)
    
    # Existing cross-reference extraction
    clean_entry, cross_references = extract_cross_references(clean_entry)
    
    # Split entry into witnesses and readings
    split_entry = witness_reading_splitter(clean_entry)
    
    # Initialize the results
    result = []
    
    if type(split_entry) is tuple:
        witnesses, manuscript_witnesses = parse_witnesses(split_entry[0])
        witnesses_result = {'Witnesses': witnesses, 'Manuscript Witnesses': manuscript_witnesses}
        if len(split_entry) == 2:
            reading = parse_reading_entry(split_entry[1])
        else:  # there are 3 groups:
            reading = parse_reading_entry(split_entry[1] + split_entry[2])
    else:
        witnesses, manuscript_witnesses = parse_witnesses(split_entry)
        witnesses_result = {'Witnesses': witnesses, 'Manuscript Witnesses': manuscript_witnesses}
        reading = ''
    
    result.append(witnesses_result)
    result.append({"Rdg": reading})
    
    if cross_references:
        result.append({"Cross References": cross_references})
    if footnotes:
        result.append({"Footnotes": footnotes})
    
    return result


def process_comma_entry(entry):
    """
    Processes a comma-separated textual entry to extract witnesses, readings, footnotes, and manuscript witnesses.

    Args:
        entry (str): The entry text to be processed.

    Returns:
        list: A list containing processed witnesses, readings, footnotes, and manuscript witnesses.
    """
    # Extract and remove footnotes
    clean_entry, footnotes = extract_footnotes(entry)
    
    # Existing cross-reference extraction
    clean_entry, cross_references = extract_cross_references(clean_entry)
    
    # Split entry into witnesses and readings
    split_entry = witness_reading_splitter(clean_entry)
    
    # Initialize the results
    result = []
    
    if type(split_entry) is tuple:
        witnesses, manuscript_witnesses = parse_comma_witnesses(split_entry[0])
        witnesses_result = {'Witnesses': witnesses, 'Manuscript Witnesses': manuscript_witnesses}
        if len(split_entry) == 2:
            reading = parse_reading_entry(split_entry[1])
        else:  # there are 3 groups:
            reading = parse_reading_entry(split_entry[1] + split_entry[2])
    else:
        witnesses, manuscript_witnesses = parse_comma_witnesses(split_entry)
        witnesses_result = {'Witnesses': witnesses, 'Manuscript Witnesses': manuscript_witnesses}
        reading = ''
    
    result.append(witnesses_result)
    result.append({"Rdg": reading})
    
    if cross_references:
        result.append({"Cross References": cross_references})
    if footnotes:
        result.append({"Footnotes": footnotes})
    
    return result

In [13]:
#processing functions, including chapter information from Appartus 3
def process_full_entry(text, chapter, previous_verse=None):
    lemma, part_entry = split_full_entry(text)
    lemma_dict = lemma_verse_processor(lemma, chapter)
    
    # Use the previous verse if the current verse list is empty
    if not lemma_dict['verses'] and previous_verse is not None:
        lemma_dict['verses'] = previous_verse
        

    # Split part_entry by '|'
    if '|' in part_entry:
        entry_parts = part_entry.split('|')
    else:
        entry_parts = [part_entry]

    # Initialize a list to hold all processed parts
    processed_parts = []

    # Process each part separately
    for part in entry_parts:
        # Split part by ',' not inside parentheses
        sub_parts = split_on_comma_not_in_parentheses(part)

        # Process each sub-part using process_comma_entry
        processed_sub_parts = [process_comma_entry(sub_part) for sub_part in sub_parts]

        # Concatenate processed sub-parts for each part
        processed_parts.append(processed_sub_parts)

    # Combine processed parts. Assuming you want them as a nested list
    decoded_entries = processed_parts

    # Return the lemma_dict and decoded_entries, along with the verses used for this entry
    return lemma_dict, decoded_entries, lemma_dict['verses']

def split_on_comma_not_in_parentheses(part):
    """
    Splits the string on ',' not inside parentheses.
    """
    sub_parts = []
    current_part = []
    paren_depth = 0  # Track depth of parentheses

    for char in part:
        if char == '(':
            paren_depth += 1
        elif char == ')':
            paren_depth -= 1
        elif char == ',' and paren_depth == 0:
            # At a top-level comma, split here
            sub_parts.append(''.join(current_part))
            current_part = []
            continue

        current_part.append(char)

    # Add the last part if there's any
    if current_part:
        sub_parts.append(''.join(current_part))

    return sub_parts

def split_full_entry(text):
    sliced_entry = text.split(sep=']')
    if len(sliced_entry) == 2:
        lemma, entry = sliced_entry
    #         print(f"lemma: {lemma}")
    #         print(f"entry: {entry}")
        return lemma, entry
    else:
        print(sliced_entry)
        return sliced_entry

# def lemma_verse_processor(text, chapter):
#     # Simplified approach: first split into digits and lemmas
#     # Regex to match the verse numbers at the beginning
#     verse_regex = r'^(\d+(?:–\d+)?)\s'
    
#     # Extract verses
#     verses_match = re.match(verse_regex, text)
#     verses = list(map(int, verses_match.group(1).split('–'))) if verses_match else []
    
#     # Isolate lemmas part by removing the verses
#     lemmas_part = text[len(verses_match.group(0)):].strip() if verses_match else text
#     return {
#         'chapter': chapter,
#         'verses': verses,
#         'lemmas': process_lemma_with_range_and_diacritics(lemmas_part)
#     }

def lemma_verse_processor(text, chapter):
    # Regex to match the verse numbers at the beginning
    verse_regex = r'^(\d+(?:–\d+)?)\s'
    
    # Extract verses
    verses_match = re.match(verse_regex, text)
    if verses_match:
        verse_range = verses_match.group(1).split('–')
        if len(verse_range) == 2 and verse_range[0] != verse_range[1]:
            verses = {'from': int(verse_range[0]), 'to': int(verse_range[1])}
        else:
            verses = int(verse_range[0])
    else:
        verses = None
    
    # Isolate lemmas part by removing the verses
    lemmas_part = text[len(verses_match.group(0)):].strip() if verses_match else text
    return {
        'chapter': chapter,
        'verses': verses,
        'lemmas': process_lemma_with_range_and_diacritics(lemmas_part)
    }


# Function to process individual lemmas or ranges, after the split,
lemma_regex = r'(k|q)?\s*([^\d\s]+)(\d?\,?\d?)'#(\d+(?:–\d+)?)\s

def process_lemma_with_range_and_diacritics(lemma):
    # Adjust regex to include diacritical marks and punctuation within Hebrew words
    
    
    # Check for range indicated by "–" and process accordingly
    if "–" in lemma:
        from_lemma, to_lemma = lemma.split("–")
        return {
            'from': process_individual_lemma(from_lemma.strip()),
            'to': process_individual_lemma(to_lemma.strip())
        }

    # Split lemma if there are separate lemmas with "/"
    split_lemmas = re.split(r'\s*/\s*', lemma) if '/' in lemma else [lemma]
    
    processed_lemmas = []
    for split_lemma in split_lemmas:
        processed = process_individual_lemma(split_lemma)
        processed_lemmas.extend(processed)
    
    return processed_lemmas

def process_individual_lemma(individual_lemma):
    matches = re.findall(lemma_regex, individual_lemma)
    processed_lemmas = []
    for match in matches:
        prefix, word, number = match
        lemma_dict = {'lemma': word}
        if prefix: lemma_dict[prefix] = True
        if number: lemma_dict['number'] = (number)
        processed_lemmas.append(lemma_dict)
    return processed_lemmas

# processing functions for sub-units of app_entry, for which there is matching lemma and verse data processed above

def extract_cross_references(text): #extract cross-references
    # Regex to match some Roman numerals: sequences of "I"s followed by an optional "V"
    pattern = r'([I]*[V]?)'
    # Find all occurrences of the pattern
    found_numerals = re.findall(pattern, text)
    # Remove empty matches from the list
    found_numerals = [numeral for numeral in found_numerals if numeral]
    # Replace found Roman numerals with an empty string
    result_text = re.sub(pattern, '', text)
    # add an extration of footnotes [\d]
    return result_text, found_numerals

def parse_witnesses(text):
    pattern = re.compile(r'\s?([^\d]*?)?(\d+)\s?\(?([^\)\d.]+)?\)?', re.DOTALL | re.UNICODE)
    parts = re.findall(pattern, text)
    # Filter out empty tuples
    return [part for part in parts if any(part)]

def post_process_witnesses(witnesses):
    # Define the set of specific values for "x"
    specific_values = {"G-B Eb ", "G-B Kb ", "G-B Msr "}  # Replace with the actual values

    # Iterate over the witnesses, except for the last one
    for i in range(len(witnesses) - 1):
        z, y, x = witnesses[i]

        # Check if "x" is one of the specific values
        if x in specific_values:
            # Remove "x" from the current tuple and prepend it to the "z" of the next witness
            witnesses[i] = (z, y, '')
            next_z, next_y, next_x = witnesses[i + 1]
            witnesses[i + 1] = (x + next_z, next_y, next_x)

    # Remove the first tuple if it becomes empty
    if witnesses and witnesses[0] == ('', '', ''):
        witnesses.pop(0)

    return witnesses

def parse_comma_witnesses(text):
    pattern = re.compile(r'\s?([^\d]*?)?(\d*)?\s?\(?([^\)\d]+)?\)?', re.DOTALL | re.UNICODE)
    parts = re.findall(pattern, text)
    # Filter out empty tuples
    return [part for part in parts if any(part)]

def parse_reading_entry(entry):
    # Refined regex pattern
    pattern = r"""
        \s?(?P<Sigla>[+<>~\.]*)                         # Captures special sigla
        \s*
        (?P<Reading>(?:[kq]*\s?)[\u0590-\u05FF\uFB1D-\uFB4F\s.]*    # Hebrew reading, including 'k', 'q'
                   (?:/\s(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*)?)  # Allows for 'k'/'q' followed by Hebrew, separated by '/'
        \s*
        \s*
        (?P<Comment>\(.*\))?                     # Captures comments
    """
    
    # Compiling regex with VERBOSE flag for better readability and explanation
    compiled_pattern = re.compile(pattern, re.VERBOSE)
    match = compiled_pattern.match(entry)

    if not match:
        return None  # Return None if no match is found

    # Extracting groups into a dictionary
    parsed_entry = {k: v for k, v in match.groupdict().items() if v}

    return parsed_entry

#splitting entry into witnesses and reading (if only one group assign to witnesses)
def witness_reading_splitter(text):
    pattern = re.compile(r'(.*?)?([\+<>~.]*\s?[kq\u0590-\u05FF]+)(.*)?', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        pattern = re.compile(r'(.*?)([\+<>~])(.*)?', re.DOTALL)
        match = pattern.match(text)
        if match:
            return match.groups()  # Returns a tuple with the three parts
        else:
            return text
        return text  # No divider matching the pattern was found


def process_entry(entry):
    clean_entry, cross_references = extract_cross_references(entry)
    split_entry = witness_reading_splitter(clean_entry)
    if type(split_entry) is tuple:
        witnesses = {'Witnesses': parse_witnesses(split_entry[0])}
        if len(split_entry) == 2:
            reading = parse_reading_entry(split_entry[1])
        else:  # there are 3 groups:
            reading = parse_reading_entry(split_entry[1] + split_entry[2])
    else:
        witnesses = {'Witnesses': parse_witnesses(split_entry)}
        reading = ''
    # Include "Cross References" only if the list is not empty
    result = [witnesses, {"Rdg": reading}]
    if cross_references:
        result.append({"Cross References": cross_references})
    # here also add the same segment just for "footnote" instead of 
    return result

def process_comma_entry(entry):
    clean_entry, cross_references = extract_cross_references(entry)
    split_entry = witness_reading_splitter(clean_entry)
    if type(split_entry) is tuple:
        witnesses = {'Witnesses': parse_comma_witnesses(split_entry[0])}
        if len(split_entry) == 2:
            reading = parse_reading_entry(split_entry[1])
        else:  # there are 3 groups:
            reading = parse_reading_entry(split_entry[1] + split_entry[2])
    else:
        witnesses = {'Witnesses': parse_comma_witnesses(split_entry)}
        reading = ''
    # Include "Cross References" only if the list is not empty
    result = [witnesses, {"Rdg": reading}]
    if cross_references:
        result.append({"Cross References": cross_references})
    return result


In [63]:
#sample entries
full_entries = [
    "Chapter 1",
    "8 k ואמאסאך / q ואמאסך] 30 93 (sm) 96 150 (pm) q IV | 93 (pm) אמסך",
    "8 שָׂך] 30 (pm) 93 (pm) 150 (pm) סךII IV (See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))",
    "6 עוד1] 30 (pm) >I II IV (similarly b. Pesaḥim 87bmss)",
    "6 הדעת1] 93 (pm) 150 (pm) דעתII",
    "14 זעקו] 30 (pm) יזעקו | 150 (pm) >",
    "10 k שעריריה / q שערוריה] 30 (pm) 89 (pm) 150 (sm) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) שערורהIV",
    "2 ללבבם] 96 (pm) ללבם | 30 93 150 (pm) בלבבםI",
    "12 עליהם] 96 (pm) להם",
    "איסירם] 30 89 איסרם | 96 (pm) אייסרם, (sm) אייסירים | 150 (pm) אסירים",
    "7 נדמֶה] 93 נדמָה | 30 (pm) + אפרים",
    "10 ואסרם] 96 (pm) יאשרם.. | 150  ..על",
    "8 רֻחמה] 30 לח..",
    "והצגתיה] 93 (pm) + כיום ערומה והצגתיה ",
    "4 דברו] 150 דברים",
    "7–8 ילכו ינקותיו – כיין לבנון] 30 > ",
    "3 כי1] 150 + לא (non voc)",
    "וכְמריו] 93 G-B Msr 34 כֹ",
    "6 בָשנה] G-B Msr 34 בָשְנָה ל ומדׄ מיש ביה בֹשְנָה",
    "k עינתם / q עונֹתם] 30 k, 89 G-B Eb 16 q, 93 96 150 (pm) עונותם",
    "10 לה] 93 (non voc) 96 150 (non voc) + את"
    
]

In [ ]:
#process lemma:
#split into digits and lemmas. then process each separately

In [738]:
[process_full_entry(example) for example in full_entries][-1]#[0]['verses']


({'verses': [10], 'lemmas': [{'lemma': 'ואסרם'}]},
 [[({'Witnesses': [('', '96', 'pm')]},
    {'Reading': {'Reading': 'יאשרם '}},
    {'Cross References': []})],
  [({'Witnesses': [('', '150', '')]},
    {'Reading': {'Reading': 'על'}},
    {'Cross References': []})]])

In [731]:
# old function, didnt take the splitting into commas consideration
# def process_full_entry(text):
#     lemma, part_entry = split_full_entry(text)
#     lemma_dict = lemma_verse_processor(lemma)
# #     if len(lemma_dict['verses'])==0: #get verse from previous entry
# #         lemma_dict['verses'] = 
        
#     #entry_units = split_entry_units # splits on | and ,
#     #for entry in entry_units:
#     decoded_entry = process_entry(part_entry)
#     return lemma_dict, decoded_entry

In [639]:
# processing functions for sub-units of app_entry, for which there is matching lemma and verse data processed above

def remove_and_list_roman_numerals(text): #extract cross-references
    # Regex to match some Roman numerals: sequences of "I"s followed by an optional "V"
    pattern = r'([I]*[V]?)'
    # Find all occurrences of the pattern
    found_numerals = re.findall(pattern, text)
    # Remove empty matches from the list
    found_numerals = [numeral for numeral in found_numerals if numeral]
    # Replace found Roman numerals with an empty string
    result_text = re.sub(pattern, '', text)
    return result_text, found_numerals

def custom_split_string(text): #process witnesses
    pattern = re.compile(r'\s?([^\d]*?)?(\d+)\s?\(?([^\)\d]+)?\)?', re.DOTALL|re.UNICODE)    
    #pattern = r'([^,\d]*?)?(\d+)\s?(\(([^\)]+)?\)?)?([\skq]?)+'
    parts = re.findall(pattern, text)
    return parts

def parse_reading_entry(entry):
    # Refined regex pattern
    pattern = r"""
        \s?(?P<Sigla>[+<>~]?)                         # Captures special sigla
        \s*
        (?P<Reading>(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*    # Hebrew reading, including 'k', 'q'
                   (?:/\s(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*)?)  # Allows for 'k'/'q' followed by Hebrew, separated by '/'
        \s*
        (?P<Comment>\(.*\))?                     # Captures comments
    """
    
    # Compiling regex with VERBOSE flag for better readability and explanation
    compiled_pattern = re.compile(pattern, re.VERBOSE)
    match = compiled_pattern.match(entry)

    if not match:
        return None  # Return None if no match is found

    # Extracting groups into a dictionary
    parsed_entry = {k: v for k, v in match.groupdict().items() if v}

    return parsed_entry

#splitting entry into witnesses and reading (if only one group assign to witnesses)
def split_string(text):
    pattern = re.compile(r'(.*?)?([\+<>]?\s?[kq\u0590-\u05FF]+)(.*)?', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        pattern = re.compile(r'(.*?)([\+<>])(.*)?', re.DOTALL)
        match = pattern.match(text)
        if match:
            return match.groups()  # Returns a tuple with the three parts
        else:
            return text
        return text  # No divider matching the pattern was found


def process_entry(entry):
    clean_entry, cross_references = remove_and_list_roman_numerals(entry)
    split_entry = split_string(clean_entry)
    if type(split_entry) is tuple:
        witnesses = {'Witnesses': custom_split_string(split_entry[0])}
        if len(split_entry)==2:
            reading = parse_reading_entry(split_entry[1])
        else: #there are 3 groups:
            reading = parse_reading_entry(split_entry[1]+split_entry[2])
    else:
        witnesses = {'Witnesses': custom_split_string(split_entry)}
        reading = ''
    return witnesses, {"Reading":reading}, {"Cross References":cross_references}


# for entry in sample_texts:
#     print(f"entry: {entry}")
#     clean_entry, cross_references = remove_and_list_roman_numerals(entry)
#     split_entry = split_string(clean_entry)
#     if type(split_entry) is tuple:
#         witnesses = {'witnesses': custom_split_string(split_entry[0])}
#         reading = parse_reading_entry(split_entry[1])
#         print(f"witnesses: {witnesses}")
#         print(f"reading: {reading}")
#     else:
#         witnesses = {'witnesses': custom_split_string(split_entry)}
#         print(f"witnesses: {witnesses}")
#     print(f"references: {cross_references}")


In [640]:
sample_texts = [
    "96 (non voc)",
    "30 (pm) 93 (pm) 150 (pm) + סךII IV (See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))",
    "93 (non voc) 96 150 (non voc) + את",
    "30 (pm) >",
    "30 + לי (non voc)I II",
    "30 (pm) >I II IV (similarly b. Pesaḥim 87bmss)",
    "93 (pm) ביהושעIV (similarly PesiqtaR 33 (153b))",
    "96 >I II IV",
    "130 k",
    "G-B Msr 34 k ממני / q ממנוIV",
    "93 כד..",
    "150 ..דברים",
    "G-B Eb 94 ותָעָד (understood as \עוד (rather than \עדי))",
    "30 89 (sm) 93 (pm) 150 (non voc) + כיI II IV"
]

In [643]:
process_entry(sample_texts[6])

({'Witnesses': [('', '93', 'pm')]},
 {'Reading': {'Reading': 'ביהושע ',
   'Comment': '(similarly PesiqtaR 33 (153b))'}},
 {'Cross References': ['IV']})

In [638]:
clean_entry, cross_references = remove_and_list_roman_numerals(sample_texts[6])
split_entry = split_string(clean_entry)
if type(split_entry) is tuple:
    witnesses = {'Witnesses': custom_split_string(split_entry[0])}
    if len(split_entry)==2:
        reading = parse_reading_entry(split_entry[1])
    else: #there are 3 groups:
        reading = parse_reading_entry(split_entry[1]+split_entry[2])

reading

{'Reading': 'ביהושע ', 'Comment': '(similarly PesiqtaR 33 (153b))'}

In [462]:
def custom_split_string(text): #process witnesses
    pattern = re.compile(r'\s?([^\d]*?)?(\d+)\s?\(?([^\)\d]+)?\)?', re.DOTALL|re.UNICODE)    
    #pattern = r'([^,\d]*?)?(\d+)\s?(\(([^\)]+)?\)?)?([\skq]?)+'
    parts = re.findall(pattern, text)
    return parts
# "96 (non voc)",
# "30 (pm) 93 (pm) 150 (pm)
test_witness = "30 (pm) 93 150 (pm)"
custom_split_string(test_witness)


[('', '30', 'pm'), ('', '93', ''), ('', '150', 'pm')]

In [623]:
#try parsing single entry app, splitting into witnesses and reading (if only one group assign to witnesses)
def split_string(text):
    pattern = re.compile(r'(.*?)?([\+<>]?\s?[kq\u0590-\u05FF]+)(.*)?', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        pattern = re.compile(r'(.*?)([\+<>])(.*)?', re.DOTALL)
        match = pattern.match(text)
        if match:
            return match.groups()  # Returns a tuple with the three parts
        else:
            return text
        return text  # No divider matching the pattern was found

# Example usage
processed_sample = [split_string(text) for text in sample_texts]

print(processed_sample)


['96 (non voc)', ('30 (pm) 93 (pm) 150 (pm) ', '+ סך', 'II IV (See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'), ('93 (non voc) 96 150 (non voc) ', '+ את', ''), ('30 (pm) ', '>', ''), ('30 ', '+ לי', ' (non voc)I II'), ('30 (pm) ', '>', 'I II IV (similarly b. Pesaḥim 87bmss)'), ('93 (pm)', ' ביהושע', 'IV (similarly PesiqtaR 33 (153b))'), ('96 ', '>', 'I II IV'), ('130', ' k', ''), ('G-B Msr 34', ' k', ' ממני / q ממנוIV'), ('93', ' כד', '..'), ('150 ..', 'דברים', ''), ('G-B Eb 94', ' ותָעָד', ' (understood as \\עוד (rather than \\עדי))'), ('30 89 (sm) 93 (pm) 150 (non voc) ', '+ כי', 'I II IV')]


In [619]:
#function for parsing the reading+comment (assumes witnesses and cross references have been removed)
def parse_reading_entry(entry):
    # Refined regex pattern
    pattern = r"""
        \s?(?P<Sigla>[+<>~]?)                         # Captures special sigla
        \s*
        (?P<Reading>(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*    # Hebrew reading, including 'k', 'q'
                   (?:/\s(?:[kq]?\s?)?[\u0590-\u05FF\uFB1D-\uFB4F\s.]*)?)  # Allows for 'k'/'q' followed by Hebrew, separated by '/'
        \s*
        (?P<Comment>\(.*\))?                     # Captures comments
    """
    
    # Compiling regex with VERBOSE flag for better readability and explanation
    compiled_pattern = re.compile(pattern, re.VERBOSE)
    match = compiled_pattern.match(entry)

    if not match:
        return None  # Return None if no match is found

    # Extracting groups into a dictionary
    parsed_entry = {k: v for k, v in match.groupdict().items() if v}

    return parsed_entry

# Process the sample reading texts with the refined function
parse_reading_entry = [parse_reading_entry(text) for text in sample_reading_texts]

parse_reading_entry


[{'Reading': 'סך ',
  'Comment': '(See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'},
 {'Sigla': '+', 'Reading': 'את'},
 {'Sigla': '>'},
 {'Sigla': '+', 'Reading': 'לי ', 'Comment': '(non voc)'},
 {'Sigla': '>', 'Comment': '(similarly b. Pesaḥim 87bmss)'},
 {'Reading': 'ביהושע ', 'Comment': '(similarly PesiqtaR 33 (153b))'},
 {},
 {'Reading': 'k'},
 {'Reading': 'k ממני / q ממנו'},
 {'Reading': 'כד..'},
 {'Reading': '..דברים'},
 {'Reading': 'נַחֵם ',
  'Comment': '(taken as infinitive, see Yeivin, Babylonian Vocalization, 1:542)'},
 {'Reading': 'חכֵם ', 'Comment': '(!)'}]

In [587]:
import re

def custom_string_processor(input_string, regex_pattern):
    # Helper function to apply regex and extract groups
    def apply_regex_and_extract(text):
        matches = re.finditer(regex_pattern, text)
        results = []
        for match in matches:
            results.append({
                'witnesses': match.group(1).strip(),
                'reading': match.group(2).strip(),
                'comments': match.group(3).strip() if match.group(3) else ''
            })
        return results

    # Process splits with "|", then ","
    def process_splits(text, delimiter):
        parts = text.split(delimiter)
        processed_parts = []
        for part in parts:
            # Apply regex to each part
            processed = apply_regex_and_extract(part)
            if processed:
                processed_parts.extend(processed)
        return processed_parts

    # Start processing
    processed_result = process_splits(input_string, '|')  # Start with the highest level of split

    return processed_result

# Custom regex pattern as provided
custom_regex = r'^(.*?)([\+<~>]?\s?[\u0590-\u05FF]+.*?)(.*)$'

# Test with the provided sample input
sample_input = "G-B msr. 30 (pm) G-A 89 (sm?) 150 (non voc) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) + שערורהIV II (bla bla (f)) | 150 >"
processed_sample = [custom_string_processor(text, custom_regex) for text in sample_texts]


print(processed_sample)

def split_string(text):
    pattern = re.compile(r'^(.*?)([\+<>]?\s?[\u0590-\u05FF]+.*?)(.*)$', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        return None  # No divider matching the pattern was found

# Example usage
# text = "G-B msr. 30 (pm) G-A 89 (pm?) 150 (sm) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) > שערורהIV (bla bla (f))"#"30 89 (sm) 93 (pm) 150 (non voc) + כיI II IV"#"93 (pm) < ביהושעIV (similarly PesiqtaR 33 (153b))"
# split_parts = split_string(text)
# if split_parts:
#     print("witnesses:", split_parts[0])
#     print("reading:", split_parts[1])
#     print("After dividers:", split_parts[2])
# else:
#     print("No dividers found.")



[[], [{'witnesses': '30 (pm) 93 (pm) 150 (pm)', 'reading': '+ סך', 'comments': 'II IV (See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'}], [{'witnesses': '93 (non voc) 96 150 (non voc)', 'reading': '+ את', 'comments': ''}], [], [{'witnesses': '30', 'reading': '+ לי', 'comments': '(non voc)I II'}], [], [{'witnesses': '93 (pm)', 'reading': 'ביהושע', 'comments': 'IV (similarly PesiqtaR 33 (153b))'}], [], [], [{'witnesses': 'G-B Msr 34 k', 'reading': 'ממני', 'comments': '/ q ממנוIV'}], [{'witnesses': '93', 'reading': 'כד', 'comments': '..'}], [{'witnesses': '150 ..', 'reading': 'דברים', 'comments': ''}], [{'witnesses': 'G-B Eb 94', 'reading': 'ותָעָד', 'comments': '(understood as \\עוד (rather than \\עדי))'}], [{'witnesses': '30 89 (sm) 93 (pm) 150 (non voc)', 'reading': '+ כי', 'comments': 'I II IV'}]]


In [37]:

def parse_apparatus_entry(entry):
    """Parse an apparatus entry into lemma(s) and content."""
    parts = entry.split(']')
    lemmas_contents = []
    for part in parts:
        if part.strip():
            lemma, content = part.split('[', 1) if '[' in part else (part, '')
            lemmas_contents.append((lemma.strip(), content.strip()))
    return lemmas_contents

def create_tei_document(apparatus_lines):
    """Create a TEI document from apparatus lines."""
    TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"
    TEI = "{%s}" % TEI_NAMESPACE
    NSMAP = {"tei": TEI_NAMESPACE}
    
    tei_root = ET.Element(TEI+"TEI", nsmap=NSMAP)
    tei_header = ET.SubElement(tei_root, TEI+"teiHeader")
    text = ET.SubElement(tei_root, TEI+"text")
    body = ET.SubElement(text, TEI+"body")
    current_chapter = None
    last_verse = None
    
    for line in apparatus_lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith('Chapter'):
            chapter_number = line.split(' ')[1]
            current_chapter = ET.SubElement(body, TEI+"div", type="chapter", n=chapter_number)
            last_verse = None
            continue
        # Use regex to check if the line starts with a verse number and capture it
        match = re.match(r"^(\d+)\s*(.*)", line)
        if match:
            verse_number, entry = match.groups()
            last_verse = verse_number
        else:
            entry = line
            verse_number = last_verse
        
        if current_chapter is not None and verse_number:
            lemmas_contents = parse_apparatus_entry(entry)
            for lemma, content in lemmas_contents:
                app = ET.SubElement(current_chapter, TEI+"app")
                lem = ET.SubElement(app, TEI+"lem", n=verse_number)
                lem.text = lemma
                if content:
                    rdg = ET.SubElement(app, TEI+"rdg")
                    rdg.text = content

    return ET.ElementTree(tei_root)

def save_tei_file(tree, filename):
    """Save the TEI XML tree to a file."""
    tree.write(filename, encoding="UTF-8", xml_declaration=True, method="xml", short_empty_elements=True)



# Replace 'your_input_file.txt' with the path to your actual input file
input_file = '01 Hosea App III - מתוקן.txt'
output_file = 'apparatus_tei.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

tei_tree = create_tei_document(lines)
save_tei_file(tei_tree, output_file)

print(f"TEI document has been saved to {output_file}.")


TEI document has been saved to apparatus_tei.xml.


In [ ]:
####### old stuff

In [47]:
import xml.etree.ElementTree as ET
import re

def create_tei_document(apparatus_lines):
    TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"
    TEI = "{%s}" % TEI_NAMESPACE
    
    tei_root = ET.Element(TEI + "TEI", xmlns=TEI_NAMESPACE)
    tei_header = ET.SubElement(tei_root, TEI + "teiHeader")
    text = ET.SubElement(tei_root, TEI + "text")
    body = ET.SubElement(text, TEI + "body")
    current_chapter = None
    last_verse_number = None
    
    for line in apparatus_lines:
        line = line.strip()
        if not line:
            continue
        if line.startswith('Chapter'):
            chapter_number = line.split(' ')[1].strip()
            current_chapter = ET.SubElement(body, TEI + "div", type="chapter", n=chapter_number)
        else:
            # Attempt to extract verse number and lemma content
            parts = re.match(r"^(\d+)\s*(.*)", line)
            if parts:
                verse_number, remainder = parts.groups()
                last_verse_number = verse_number  # Update last verse number with current
                
                # Further split to separate lemma from variants, if present
                lemma_section, variants_section = remainder.split(']', 1) if ']' in remainder else (remainder, "")
                lemma_section = lemma_section.strip()
                variants_section = variants_section.strip()

                if current_chapter is not None and verse_number:
                    # Create an apparatus entry for the lemma
                    app = ET.SubElement(current_chapter, TEI + "app")
                    lem = ET.SubElement(app, TEI + "lem", n=verse_number)
                    lem.text = lemma_section
                    
                    # Add variant readings if present
                    if variants_section:
                        rdg = ET.SubElement(app, TEI + "rdg")
                        rdg.text = variants_section
            else:
                print(f"Line does not conform to expected format: {line}")

    return ET.ElementTree(tei_root)

def save_tei_file(tree, filename):
    tree.write(filename, encoding="UTF-8", xml_declaration=True, method="xml", short_empty_elements=True)

# Replace 'your_input_file.txt' with the path to your actual input file
input_file = '01 Hosea App III - מתוקן.txt'
output_file = 'apparatus_tei.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

tei_tree = create_tei_document(lines)
save_tei_file(tei_tree, output_file)

print(f"TEI document has been saved to {output_file}.")


Line does not conform to expected format: ﻿App III: Hosea
Line does not conform to expected format: יחזקיה] 30 93 (pm) 96 יחזקיהו
Line does not conform to expected format: ירבעם בן] 30 + נבט (non voc)
Line does not conform to expected format: לו] 96 >I  II IV
Line does not conform to expected format: ממלכוּת] 96 ממלכוֹת
Line does not conform to expected format: יזרעאל] 150 ישראל (parall; but 150-Tg: יזרעאל)
Line does not conform to expected format: כי] 93 (pm) + את
Line does not conform to expected format: אוסיף] 150 (pm) >
Line does not conform to expected format: את] 93 (pm) >
Line does not conform to expected format: בסוסים] 96 ובסוסיםI IV
Line does not conform to expected format: אשר2] 96 + לא
Line does not conform to expected format: אחד] 30 (pm) 150 (pm) >
Line does not conform to expected format: והצגתיה] 93 (pm) + כיום ערומה והצגתיה
Line does not conform to expected format: ושתִּה] 150 (pm) ושמתיה
Line does not conform to expected format: כי] 150 (non voc) + כי
Line does not co

In [10]:
import xml.etree.ElementTree as ET
import re


def create_apparatus_entry(verse_number, content, TEI):
    """Create TEI element for an apparatus entry."""
    app = ET.Element(TEI + "app")
    
    # Extract lemma text and the rest (witnesses, variant reading, and comments)
    lemma_text, _, rest = content.partition(']')
    lem = ET.SubElement(app, TEI + "lem")
    lem.text = lemma_text.strip()
    
    # Extract comments
    comments = re.findall(r'\((.*?)\)', rest)
    for comment in comments:
        note = ET.SubElement(app, TEI + "note")
        note.text = comment
    
    # Remove comments from rest for further processing
    rest = re.sub(r'\(.*?\)', '', rest).strip()
    
    # Extract and process witnesses and cross-references
    if rest:
        rdg = ET.SubElement(app, TEI + "rdg")
        witnesses, _, variant_reading = rest.partition(' ')
        if witnesses:
            rdg.set('wit', witnesses.strip())
        if variant_reading:
            rdg.text = variant_reading.strip()
        
        # Extract cross-references, assuming they are indicated by Roman numerals at the start
        cross_refs = re.findall(r'\bI{1,3}V?|\bIV', rest)
        for ref in cross_refs:
            ref_element = ET.SubElement(rdg, TEI + "ref")
            ref_element.set('target', '#' + ref)  # Assuming target IDs are prefixed with '#'
            ref_element.text = "See apparatus entry " + ref
    
    return app

def create_tei_document(apparatus_lines):
    TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"
    TEI = "{%s}" % TEI_NAMESPACE
    root = ET.Element(TEI + "TEI", xmlns=TEI_NAMESPACE)
    header = ET.SubElement(root, TEI + "teiHeader")
    text = ET.SubElement(root, TEI + "text")
    body = ET.SubElement(text, TEI + "body")
    div = ET.SubElement(body, TEI + "div")
    
    last_verse_number = None
    for line in apparatus_lines:
        line = line.strip()
        if not line:
            continue
        
        # Determine if the line starts with a verse number
        match = re.match(r'^(\d+)', line)
        if match:
            last_verse_number = match.group(1)
            content = line[len(last_verse_number):].strip()
        else:
            content = line
        
        if last_verse_number:
            entry = create_apparatus_entry(last_verse_number, content, TEI)
            div.append(entry)
    
    return ET.ElementTree(root)

def save_tei_file(tree, filename):
    tree.write(filename, encoding="UTF-8", xml_declaration=True, method="xml")


# Replace 'your_input_file.txt' with the path to your actual input file
input_file = '01 Hosea App III - מתוקן.txt'
output_file = 'apparatus_tei.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

tei_tree = create_tei_document(lines)
save_tei_file(tei_tree, output_file)

print(f"TEI document has been saved to {output_file}.")


TEI document has been saved to apparatus_tei.xml.


In [232]:
#split entry into witnesses, reading, and comments 

def split_string(text):
    pattern = re.compile(r'^(.*?)([\+<>]?\s?[\u0590-\u05FF]+.*?)(.*)$', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()  # Returns a tuple with the three parts
    else:
        return None  # No divider matching the pattern was found

# Example usage
text = "G-B msr. 30 (pm) G-A 89 (pm?) 150 (sm) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) > שערורהIV (bla bla (f))"#"30 89 (sm) 93 (pm) 150 (non voc) + כיI II IV"#"93 (pm) < ביהושעIV (similarly PesiqtaR 33 (153b))"
split_parts = split_string(text)
if split_parts:
    print("witnesses:", split_parts[0])
    print("reading:", split_parts[1])
    print("After dividers:", split_parts[2])
else:
    print("No dividers found.")


witnesses: G-B msr. 30 (pm) G-A 89 (pm?) 150 (sm) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) 
reading: > שערורה
After dividers: IV (bla bla (f))


[('G-B msr. ', '30', 'pm'), ('G-A ', '89', 'pm?'), ('', '150', 'non voc'), ('', '30', 'sm'), ('', '89', 'sm'), ('', '93', 'sm'), ('MS-G ', '150', 'pm')]


In [161]:
#parse comments
import re

def remove_and_list_roman_numerals(text):
    # Regex to match some Roman numerals: sequences of "I"s followed by an optional "V"
    pattern = r'([I]*[V]?)'
    # Find all occurrences of the pattern
    found_numerals = re.findall(pattern, text)
    # Remove empty matches from the list
    found_numerals = [numeral for numeral in found_numerals if numeral]
    # Replace found Roman numerals with an empty string
    result_text = re.sub(pattern, '', text)
    return result_text, found_numerals

# Example usage
text = "II IV (See b. R.HaŠanamss 23b, LamR Buber 1:16 (40b))"
result_text, numerals_found = remove_and_list_roman_numerals(text)
print("Modified text:", result_text.strip())
print("Numerals found:", numerals_found)



Modified text: (See b. R.HaŠanamss 23b, LamR Buber 1:16 (40b))
Numerals found: ['II', 'IV']


In [178]:
def process_entry(entry):
    split_parts = split_string(entry)
    if split_parts is None:
        return None
    
    witnesses, reading, comments = split_parts

    structured_entry = {
        'witnesses': [],
        'reading': reading,
        'comments': '',
        'cross_references': []
    }

    for part in custom_split_string(witnesses):
        # Assuming part[1] contains the witness number and part[0], part[2], part[3] contain additional info
        witness_info = {
            'n': part[1],
            'text': f"{part[0]}{part[1]} {part[2].strip()}{part[3]}"
        }
        structured_entry['witnesses'].append(witness_info)

    comments_text, numerals_found = remove_and_list_roman_numerals(comments)
    structured_entry['comments'] = comments_text
    structured_entry['cross_references'] = numerals_found

    return structured_entry

def create_apparatus_entry(verse_number, content, TEI):
    """Create TEI element for an apparatus entry."""
    TEI_ns = {'tei': TEI}  # Define the namespace dictionary if needed
    app = ET.Element(f"{{{TEI}}}app")  # Using namespace in the tag
    
    # Extract lemma text and the rest (witnesses, variant reading, and comments)
    lemma_text, _, rest = content.partition(']')
    lem = ET.SubElement(app, f"{{{TEI}}}lem")
    lem.text = lemma_text.strip('[] ')

    structured_entry = process_entry(rest)
    if not structured_entry:
        return None

    for witness in structured_entry['witnesses']:
        wit_element = ET.SubElement(app, f"{{{TEI}}}wit", {'n': witness['n']})
        wit_element.text = witness['text']
    
    rdg_element = ET.SubElement(app, f"{{{TEI}}}rdg")
    rdg_element.text = structured_entry['reading']

    if structured_entry['comments']:
        comment_element = ET.SubElement(app, f"{{{TEI}}}note")
        comment_element.text = structured_entry['comments']

    for ref in structured_entry['cross_references']:
        ref_element = ET.SubElement(app, f"{{{TEI}}}ref")
        ref_element.text = ref

    return app


In [181]:
import xml.etree.ElementTree as ET
import re


def create_tei_document(apparatus_lines):
    TEI_NAMESPACE = "http://www.tei-c.org/ns/1.0"
    ET.register_namespace('', TEI_NAMESPACE)  # Register the default namespace

    # Create the root element without redundantly specifying the xmlns attribute
    root = ET.Element("{%s}TEI" % TEI_NAMESPACE)
    header = ET.SubElement(root, "{%s}teiHeader" % TEI_NAMESPACE)
    text = ET.SubElement(root, "{%s}text" % TEI_NAMESPACE)
    body = ET.SubElement(text, "{%s}body" % TEI_NAMESPACE)
    div = ET.SubElement(body, "{%s}div" % TEI_NAMESPACE)
    
    last_verse_number = None
    for line in apparatus_lines:
        line = line.strip()
        if not line:
            continue
        
        # Determine if the line starts with a verse number
        match = re.match(r'^(\d+)', line)
        if match:
            last_verse_number = match.group(1)
            content = line[len(last_verse_number):].strip()
        else:
            content = line
        
        if last_verse_number:
            entry = create_apparatus_entry(last_verse_number, content, TEI_NAMESPACE)
            if entry is not None:  # Ensure entry creation was successful
                div.append(entry)
    
    return ET.ElementTree(root)

def save_tei_file(tree, filename):
    tree.write(filename, encoding="UTF-8", xml_declaration=True, method="xml")


# Replace 'your_input_file.txt' with the path to your actual input file
input_file = '01 Hosea App III - מתוקן.txt'
output_file = 'apparatus_tei.xml'

with open(input_file, 'r', encoding='utf-8') as f:
    lines = f.readlines()

tei_tree = create_tei_document(lines)
save_tei_file(tei_tree, output_file)

print(f"TEI document has been saved to {output_file}.")


TEI document has been saved to apparatus_tei.xml.


In [227]:
import re

def split_string(text):
    pattern = re.compile(r'^(.*?)([\+<~>]?\s?[\u0590-\u05FF]+.*?)(.*)$', re.DOTALL)
    match = pattern.match(text)
    if match:
        return match.groups()
    else:
        return None

def custom_split_string(text):
    pattern = re.compile(r'([^,\d]*?)?(\d+)\s?(\([^\)]+\)?)?([\skq]*)?', re.DOTALL|re.UNICODE)
    parts = re.findall(pattern, text)
    return parts

def remove_and_list_roman_numerals(text):
    pattern = r'([I]*[V]?)'
    found_numerals = re.findall(pattern, text)
    found_numerals = [numeral for numeral in found_numerals if numeral]
    result_text = re.sub(pattern, '', text)
    return result_text, found_numerals

# def process_entry(entry):
#     split_parts = split_string(entry)
#     if split_parts is None:
#         return "Unable to process entry: No valid dividers found."
    
#     witnesses, reading, comments = split_parts

#     witness_entries = []
#     for part in custom_split_string(witnesses):
#         witness_entry = f'<witness n="{part[1]}">{part[0]}{part[1]} {part[2].strip()}{part[3]}</witness>'
#         witness_entries.append(witness_entry)
#     witnesses_tagged = "\n".join(witness_entries)

#     reading_tagged = f'<reading>{reading}</reading>'

#     comments_text, numerals_found = remove_and_list_roman_numerals(comments)
#     comments_tagged = f'<comment>{comments_text}</comment>'
#     cross_references = "\n".join([f'<ref>{numeral}</ref>' for numeral in numerals_found])

#     # Combine all parts, placing cross_references outside the comment
#     tei_entry = f"{witnesses_tagged}\n{reading_tagged}\n{comments_tagged}\n{cross_references}"
#     return tei_entry

# Example usage
entry ="G-B msr. 30 (pm) G-A 89 (pm?) 150 (non voc) k, 30 (sm) 89 (sm) 93 (sm) 96 150 (pm) q, 93 (pm) > שערורהIV II (bla bla (f))"
processed_entry = process_entry(entry)
print(processed_entry)


{'witnesses': [{'n': '30', 'text': 'G-B msr. 30 (pm) '}, {'n': '89', 'text': 'G-A 89 (pm?) '}, {'n': '150', 'text': '150 (non voc) k'}, {'n': '30', 'text': ' 30 (sm) '}, {'n': '89', 'text': '89 (sm) '}, {'n': '93', 'text': '93 (sm) '}, {'n': '96', 'text': '96 '}, {'n': '150', 'text': '150 (pm) q'}, {'n': '93', 'text': ' 93 (pm) '}], 'reading': '> שערורה', 'comments': '  (bla bla (f))', 'cross_references': ['IV', 'II']}


In [ ]:
#### 

In [ ]:
[' ', '"', '$', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', '<', '=', '>', 'E', 'I', 'T', '_', 'a', 'b', 'c', 'd', 'e', 'g', 'h', 'i', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', '֑', '֔', '֕', '֖', '֗', '֙', '֛', '֜', '֞', '֣', '֤', '֥', '֨', '֩', 'ְ', 'ֱ', 'ֲ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֻ', 'ּ', 'ֽ', '׀', 'ׁ', 'ׂ']

In [17]:
#old function. didnt handle the "E-G misr" etc.
def compile_and_display_entries(entries):
    previous_verse = None
    compiled_results = []

    for entry in entries:
        lemma_dict, decoded_entries, used_verse = process_full_entry(entry, previous_verse)
        previous_verse = used_verse

        # Combine the lemma_dict and decoded_entries for display
        entry_result = {
            'Lemma': lemma_dict,
            'Decoded Entries': decoded_entries
        }
        compiled_results.append(entry_result)

    # Display the results in a structured format
    for result in compiled_results:
        print("Lemma:")
        for key, value in result['Lemma'].items():
            print(f"  {key}: {value}")
        print("Decoded Entries:")
        last_witness_second_group = None
        for i, decoded_entries_list in enumerate(result['Decoded Entries']):
            for j, decoded_entry in enumerate(decoded_entries_list):
                variant_type = "Variant" if j == 0 else "Related Variant"
                print(f"  {variant_type}:")
                for item in decoded_entry:
                    if isinstance(item, dict) and 'Witnesses' in item:
                        # Update witnesses with last non-empty second group if needed
                        updated_witnesses = []
                        for witness in item['Witnesses']:
                            if witness[1] == '' and last_witness_second_group:
                                updated_witnesses.append((witness[0], last_witness_second_group, witness[2]))
                            else:
                                updated_witnesses.append(witness)
                                if witness[1]:
                                    last_witness_second_group = witness[1]
                        item['Witnesses'] = updated_witnesses
                    if isinstance(item, dict):
                        for key, value in item.items():
                            print(f"    {key}: {value}")
                    else:
                        print(f"    {item}")
        print("-" * 50)  # Separator line

# Example usage
compile_and_display_entries(full_entries)


Lemma:
  verses: [8]
  lemmas: [{'lemma': 'ואמאסאך', 'k': True}, {'lemma': 'ואמאסך', 'q': True}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', ''), ('', '93', 'sm'), ('', '96', ''), ('', '150', 'pm')]
    Reading: {'Reading': 'q  '}
    Cross References: ['IV']
  Variant:
    Witnesses: [('', '93', 'pm')]
    Reading: {'Reading': 'אמסך'}
--------------------------------------------------
Lemma:
  verses: [8]
  lemmas: [{'lemma': 'שָׂך'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', 'pm'), ('', '93', 'pm'), ('', '150', 'pm')]
    Reading: {'Reading': 'סך  ', 'Comment': '(See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'}
    Cross References: ['II', 'IV']
--------------------------------------------------
Lemma:
  verses: [6]
  lemmas: [{'lemma': 'עוד', 'number': '1'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', 'pm')]
    Reading: {'Sigla': '>', 'Comment': '(similarly b. Pesaḥim 87bmss)'}
    Cross References: ['I', 'II', 'IV']
----------------------------

In [39]:
#old processing that doesnt include chapter extraction
def post_process_witnesses(witnesses):
    # Define the set of specific values for "x"
    specific_values = {"G-B Eb ", "G-B Kb ", "G-B Msr "}  # Replace with the actual values

    # Iterate over the witnesses, except for the last one
    for i in range(len(witnesses) - 1):
        z, y, x = witnesses[i]

        # Check if "x" is one of the specific values
        if x in specific_values:
            # Remove "x" from the current tuple and prepend it to the "z" of the next witness
            witnesses[i] = (z, y, None)
            next_z, next_y, next_x = witnesses[i + 1]
            witnesses[i + 1] = (x + next_z, next_y, next_x)

    # Remove the first tuple if it becomes empty
    if witnesses and witnesses[0] == (None, None, None):
        witnesses.pop(0)

    return witnesses

# Example usage
def compile_and_display_entries(entries):
    previous_verse = None
    compiled_results = []

    for entry in entries:
        lemma_dict, decoded_entries, used_verse = process_full_entry(entry, previous_verse)
        previous_verse = used_verse

        # Combine the lemma_dict and decoded_entries for display
        entry_result = {
            'Lemma': lemma_dict,
            'Decoded Entries': decoded_entries
        }
        compiled_results.append(entry_result)

    # Display the results in a structured format
    for result in compiled_results:
        print("Lemma:")
        for key, value in result['Lemma'].items():
            print(f"  {key}: {value}")
        print("Decoded Entries:")
        last_witness_second_group = None
        for i, decoded_entries_list in enumerate(result['Decoded Entries']):
            for j, decoded_entry in enumerate(decoded_entries_list):
                variant_type = "Variant" if j == 0 else "Related Variant"
                print(f"  {variant_type}:")
                for item in decoded_entry:
                    if isinstance(item, dict) and 'Witnesses' in item:
                        # Update witnesses with last non-empty second group if needed
                        updated_witnesses = []
                        for witness in item['Witnesses']:
                            if witness[1] == '' and last_witness_second_group:
                                updated_witnesses.append((witness[0], last_witness_second_group, witness[2]))
                            else:
                                updated_witnesses.append(witness)
                                if witness[1]:
                                    last_witness_second_group = witness[1]
                        # Apply post-processing to witnesses
                        item['Witnesses'] = post_process_witnesses(updated_witnesses)
                    if isinstance(item, dict):
                        for key, value in item.items():
                            print(f"    {key}: {value}")
                    else:
                        print(f"    {item}")
        print("-" * 50)  # Separator line
    return compiled_results
# Example usage
test = compile_and_display_entries(full_entries)


Lemma:
  verses: [8]
  lemmas: [{'lemma': 'ואמאסאך', 'k': True}, {'lemma': 'ואמאסך', 'q': True}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', ''), ('', '93', 'sm'), ('', '96', ''), ('', '150', 'pm')]
    Reading: {'Reading': 'q  '}
    Cross References: ['IV']
  Variant:
    Witnesses: [('', '93', 'pm')]
    Reading: {'Reading': 'אמסך'}
--------------------------------------------------
Lemma:
  verses: [8]
  lemmas: [{'lemma': 'שָׂך'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', 'pm'), ('', '93', 'pm'), ('', '150', 'pm')]
    Reading: {'Reading': 'סך  ', 'Comment': '(See b. R.HaŠanamss 23b, (LamR) Buber 1:16 (40b))'}
    Cross References: ['II', 'IV']
--------------------------------------------------
Lemma:
  verses: [6]
  lemmas: [{'lemma': 'עוד', 'number': '1'}]
Decoded Entries:
  Variant:
    Witnesses: [('', '30', 'pm')]
    Reading: {'Sigla': '>', 'Comment': '(similarly b. Pesaḥim 87bmss)'}
    Cross References: ['I', 'II', 'IV']
----------------------------